<a href="https://colab.research.google.com/github/KyoungmiKwon/Bigdata_Training_at_ITwill/blob/main/Team_Project/C05_JY_word2vec_softmax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

In [ ]:
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
%%bash
bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
pip3 install /tmp/mecab-python-0.996

In [ ]:
import konlpy
from konlpy.tag import Mecab

In [ ]:
mecab=Mecab()

In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
from gensim.models import word2vec

In [ ]:

from gensim.models import Doc2Vec

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
url='https://github.com/KyoungmiKwon/Bigdata_Training_at_ITwill/raw/main/Team_Project/C01_KM_Data_Labeling_resized.xlsx'
news=pd.read_excel(url,header=5,usecols=['Code','Date','Journal','Title','Text','URL','Sentiment'])

In [ ]:
news

In [ ]:
news=news.drop_duplicates(['Date'],keep='first')#첫번째 중복값 놔둠ews=news.drop_duplicates(['Date'],keep='first')#첫번째 중복값 놔둠

In [ ]:
news

In [ ]:
for i in range(len(news)):
  news.iloc[i]['Text']=news.iloc[i]['Text'].split(sep='@')[0]

In [ ]:
news.to_excel("articles.xlsx")
news = pd.read_excel('/content/articles.xlsx', header=0, usecols=['Code', 'Date', 'Journal', 'Title','Text', 'Sentiment'])

In [ ]:
news


In [ ]:
news[:3]

In [ ]:
tag_classes=['NNP','NNG','VV+EC','XSV+EP']#추출할 품사 종류 
stopwords=[np.nan,'삼성전자','삼성','인텔','데일리','기자','LG전자','LG','YG','JYP','뉴스','파이낸셜']
Text=[]
for main in news['Text']:
  temp=[]
  for word in mecab.pos(main): #mecab.post(단어, 품사 )형태의 word뽑아내서 하나씩 사용 
    if word[1] in tag_classes:#word[1] 품사가 추출할 품사 종류에 들어 있으면 
      if word[0] not in stopwords: #불용어에 들어 있지 않으면
        temp.append(word[0]) #단어=>word추가 
  Text.append(temp) #test셋 배열에 저장

In [ ]:
len(Text)

In [ ]:
Text

워드투벡터 인코딩 하기 위해서 
단어, 품사 따로 저장 필요 
왜 단어랑 태깅 정보를 따로 저장해야하지?

sentences, pos_tags = [], [] 
for tagged_sentence in tagged_sentences: # 3,914개의 문장 샘플을 1개씩 불러온다.
    sentence, tag_info = zip(*tagged_sentence) # 각 샘플에서 단어들은 sentence에 품사 태깅 정보들은 tag_info에 저장한다.
    sentences.append(list(sentence)) # 각 샘플에서 단어 정보만 저장한다.
    pos_tags.append(list(tag_info)) # 각 샘플에서 품사 태깅 정보만 저장한다.


#학습시 필요한 하이퍼 파라미터
size = 워드 벡터의 특징 값. 즉, 임베딩 된 벡터의 차원.
window = 컨텍스트 윈도우 크기
min_count = 단어 최소 빈도 수 제한 (빈도가 적은 단어들은 학습하지 않는다.)
workers = 학습을 위한 프로세스 수
sg = 0은 CBOW, 1은 Skip-gram.


In [ ]:
num_features=1500 #워드 벡터 특정 값 수 
min_word_count=2
num_workers=4
context=5
downsampling=1e-3

In [ ]:

model = word2vec.Word2Vec(Text,
                        workers=num_workers,
                        size=num_features,
                        min_count=min_word_count,
                        window=context,
                        sample=downsampling) #epoch= 횟수 추가할수있음

In [ ]:
model_name="word2_model"
model.save(model_name)

In [ ]:
def get_features(words, model, num_features):
    # 출력 벡터 초기화
    feature_vector = np.zeros((num_features), dtype=np.float32)
    
    num_words = 0
    # 어휘 사전 준비
    index2word_set = set(model.wv.index2word)
    
    for w in words:
        if w in index2word_set:
            num_words = 1
            # 사전에 해당하는 단어에 대해 단어 벡터를 더함
            feature_vector = np.add(feature_vector, model[w])
            
    # 문장의 단어 수만큼 나누어 단어 벡터의 평균값을 문장 벡터로 함
    feature_vector = np.divide(feature_vector, num_words)
    return feature_vector

In [ ]:
def get_dataset(news, model, num_features):
    dataset = list()
    
    for s in news:
        dataset.append(get_features(s, model, num_features))
        
    NewsFeatureVecs = np.stack(dataset)
    return NewsFeatureVecs 

In [ ]:
test_data_vecs = get_dataset(Text, model, num_features)

In [ ]:
len(test_data_vecs)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X=test_data_vecs
y=news['Sentiment'].values
#y종속변수 x독립변수수
#두 관계가 서로 영향을 미치나? 
#학생의 공부 시간에 따라서 다음과 같은 점수를 얻었다는 데이터
#2시간 공부하면 25점
X_train,X_test,y_train,y_test=train_test_split(X,y,
                                               test_size=0.2,
                                               random_state=42)

from tensorflow.keras.utils import to_categorical
#훈련셋 셰잎 3개로 맞춰줘야 소프트맥스 함수적용가능
y_train = to_categorical(y_train,3)
y_test = to_categorical(y_test,3)

# 훈련 데이터와 테스트 데이터에 대해서 원-핫 인코딩
print(y_train[:5])
print(y_test[:5])

In [ ]:
print(y_train[:5])
print(y_test[:5])

In [ ]:
print("라벨링종류",news['Sentiment'].unique())

In [ ]:
print("벡터화된단어",X)

첫번째 인자 : 출력 뉴런의 수를 설정합니다.
input_dim : 입력 뉴런의 수를 설정합니다.
init : 가중치 초기화 방법 설정합니다.
‘uniform’ : 균일 분포
‘normal’ : 가우시안 분포
activation : 활성화 함수 설정합니다.
‘linear’ : 디폴트 값, 입력뉴런과 가중치로 계산된 결과값이 그대로 출력으로 나옵니다.
‘relu’ : rectifier 함수, 은익층에 주로 쓰입니다.
‘sigmoid’ : 시그모이드 함수, 이진 분류 문제에서 출력층에 주로 쓰입니다.
‘softmax’ : 소프트맥스 함수, 다중 클래스 분류 문제에서 출력층에 주로 쓰입니다.

In [ ]:
import seaborn as sns 
sns.set(style="ticks",color_codes=True)
g=sns.pairplot(news,hue="Sentiment",palette="husl")

In [ ]:
import matplotlib.pyplot as plt
news['Sentiment'].value_counts().plot(kind='bar')

In [ ]:
from tensorflow.keras.models import Sequential # 케라스의 Sequential()을 임포트
from tensorflow.keras.layers import Dense # 케라스의 Dense()를 임포트
from tensorflow.keras import optimizers # 케라스의 옵티마이저를 임포트

model=Sequential()
model.add(Dense(3,  activation='softmax'))
sgd=optimizers.SGD(lr=0.01)
# 학습률(learning rate, lr)은 0.01로 합니다.
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
#다중클래스 분류문제는 'categofical_crossentropy'
# 옵티마이저는 경사하강법의 일종인 adam을 사용합니다.
# 손실 함수(Loss function)는 크로스 엔트로피 함수를 사용합니다.
history=model.fit(X_train,y_train, batch_size=1, epochs=200, validation_data=(X_test, y_test))
# 주어진 X와 y데이터에 대해서 오차를 최소화하는 작업을 200번 시도합니다.

accuracy:훈련데이터 정확도
val_acc :테스트데이터 정확도 

에포크에 따른 정확도 출력

In [ ]:
epochs = range(1, len(history.history['accuracy']) + 1)
plt.plot(epochs, history.history['loss'])
plt.plot(epochs, history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
print("\n 테스트 정확도: %.4f" % (model.evaluate(X_test, y_test)[1]))